In [ ]:
from collections import Counter
import sklearn
import random
import pandas as pd

from sklearn.model_selection import StratifiedKFold

from pandas import read_csv
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import numpy as np
from sklearn.base import is_regressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble.forest import BaseForest
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree.tree import BaseDecisionTree
from sklearn.utils import check_random_state
from sklearn.utils import check_X_y
#from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import os
from pycm import *
import statistics
from scipy.stats.mstats import gmean
from random import randint
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
class SMOTE(object):
    """Implementation of Synthetic Minority Over-Sampling Technique (SMOTE).
    SMOTE performs oversampling of the minority class by picking target 
    minority class samples and their nearest minority class neighbors and 
    generating new samples that linearly combine features of each target 
    sample with features of its selected minority class neighbors [1].
    Parameters
    ----------
    k_neighbors : int, optional (default=5)
        Number of nearest neighbors.
    random_state : int or None, optional (default=None)
        If int, random_state is the seed used by the random number generator.
        If None, the random number generator is the RandomState instance used
        by np.random.
    References
    ----------
    .. [1] N. V. Chawla, K. W. Bowyer, L. O. Hall, and P. Kegelmeyer. "SMOTE:
           Synthetic Minority Over-Sampling Technique." Journal of Artificial
           Intelligence Research (JAIR), 2002.
    """

    def __init__(self, k_neighbors=3, random_state=None):
        self.k = k_neighbors
        self.random_state = random_state

    def sample(self, n_samples):
        """Generate samples.
        Parameters
        ----------
        n_samples : int
            Number of new synthetic samples.
        Returns
        -------
        S : array, shape = [n_samples, n_features]
            Returns synthetic samples.
        """
        np.random.seed(seed=self.random_state)

        S = np.zeros(shape=(n_samples, self.n_features))
        # Calculate synthetic samples.
        for i in range(n_samples):
            j = np.random.randint(0, self.X.shape[0])

            # Find the NN for each sample.
            # Exclude the sample itself.
            nn = self.neigh.kneighbors(self.X[j].reshape(1, -1),
                                       return_distance=False)[:, 1:]
            nn_index = np.random.choice(nn[0])

            dif = self.X[nn_index] - self.X[j]
            gap = np.random.random()

            S[i, :] = self.X[j, :] + gap * dif[:]

        return S

    def fit(self, X):
        """Train model based on input data.
        Parameters
        ----------
        X : array-like, shape = [n_minority_samples, n_features]
            Holds the minority samples.
        """
        self.X = X
        self.n_minority_samples, self.n_features = self.X.shape

        # Learn nearest neighbors.
        self.neigh = NearestNeighbors(n_neighbors=self.k + 1)
        self.neigh.fit(self.X)

        return self


class SMOTEBoost(AdaBoostClassifier):
    """Implementation of SMOTEBoost.
    SMOTEBoost introduces data sampling into the AdaBoost algorithm by
    oversampling the minority class using SMOTE on each boosting iteration [1].
    This implementation inherits methods from the scikit-learn 
    AdaBoostClassifier class, only modifying the `fit` method.
    Parameters
    ----------
    n_samples : int, optional (default=100)
        Number of new synthetic samples per boosting step.
    k_neighbors : int, optional (default=5)
        Number of nearest neighbors.
    base_estimator : object, optional (default=DecisionTreeClassifier)
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper `classes_`
        and `n_classes_` attributes.
    n_estimators : int, optional (default=50)
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.
    learning_rate : float, optional (default=1.)
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.
    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.
    random_state : int or None, optional (default=None)
        If int, random_state is the seed used by the random number generator.
        If None, the random number generator is the RandomState instance used
        by np.random.
    References
    ----------
    .. [1] N. V. Chawla, A. Lazarevic, L. O. Hall, and K. W. Bowyer.
           "SMOTEBoost: Improving Prediction of the Minority Class in
           Boosting." European Conference on Principles of Data Mining and
           Knowledge Discovery (PKDD), 2003.
    """

    def __init__(self,
                 n_samples=100,
                 k_neighbors=5,
                 base_estimator=None,
                 n_estimators=100,
                 learning_rate=1.,
                 algorithm='SAMME.R',
                 random_state=None):

        self.n_samples = n_samples
        self.algorithm = algorithm
        self.smote = SMOTE(k_neighbors=k_neighbors,
                           random_state=random_state)        
        
        super(SMOTEBoost, self).__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state)
        super(AdaBoostClassifier, self).__init__(
            DecisionTreeClassifier(max_depth=5)
        )

    def fit(self, X, y, sample_weight=None, minority_target=None):
        """Build a boosted classifier/regressor from the training set (X, y),
        performing SMOTE during each boosting step.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrix can be CSC, CSR, COO,
            DOK, or LIL. COO, DOK, and LIL are converted to CSR. The dtype is
            forced to DTYPE from tree._tree if the base classifier of this
            ensemble weighted boosting classifier is a tree or forest.
        y : array-like of shape = [n_samples]
            The target values (class labels in classification, real numbers in
            regression).
        sample_weight : array-like of shape = [n_samples], optional
            Sample weights. If None, the sample weights are initialized to
            1 / n_samples.
        minority_target : int
            Minority class label.
        Returns
        -------
        self : object
            Returns self.
        Notes
        -----
        Based on the scikit-learn v0.18 AdaBoostClassifier and
        BaseWeightBoosting `fit` methods.
        """
        # Check that algorithm is supported.
        if self.algorithm not in ('SAMME', 'SAMME.R'):
            raise ValueError("algorithm %s is not supported" % self.algorithm)

        # Check parameters.
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero")

        if (self.base_estimator is None or
                isinstance(self.base_estimator, (BaseDecisionTree,
                                                 BaseForest))):
            DTYPE = np.float64  # from fast_dict.pxd
            dtype = DTYPE
            accept_sparse = 'csc'
        else:
            dtype = None
            accept_sparse = ['csr', 'csc']

        X, y = check_X_y(X, y, accept_sparse=accept_sparse, dtype=dtype,
                         y_numeric=is_regressor(self))

        if sample_weight is None:
            # Initialize weights to 1 / n_samples.
            sample_weight = np.empty(X.shape[0], dtype=np.float64)
            sample_weight[:] = 1. / X.shape[0]
        else:
            sample_weight = check_array(sample_weight, ensure_2d=False)
            # Normalize existing weights.
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

            # Check that the sample weights sum is positive.
            if sample_weight.sum() <= 0:
                raise ValueError(
                    "Attempting to fit with a non-positive "
                    "weighted number of samples.")

        if minority_target is None:
            # Determine the minority class label.
            stats_c_ = Counter(y)
            maj_c_ = max(stats_c_, key=stats_c_.get)
            min_c_ = min(stats_c_, key=stats_c_.get)
            self.minority_target = min_c_
        else:
            self.minority_target = minority_target

        # Check parameters.
        self._validate_estimator()

        # Clear any previous fit results.
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float64)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float64)

        random_state = check_random_state(self.random_state)

        for iboost in range(self.n_estimators):
            # SMOTE step.
            X_min = X[np.where(y == self.minority_target)]
            self.smote.fit(X_min)
            X_syn = self.smote.sample(self.n_samples)
            y_syn = np.full(X_syn.shape[0], fill_value=self.minority_target,
                            dtype=np.int64)

            # Normalize synthetic sample weights based on current training set.
            sample_weight_syn = np.empty(X_syn.shape[0], dtype=np.float64)
            sample_weight_syn[:] = 1. / X.shape[0]

            # Combine the original and synthetic samples.
            X = np.vstack((X, X_syn))
            y = np.append(y, y_syn)

            # Combine the weights.
            sample_weight = \
                np.append(sample_weight, sample_weight_syn).reshape(-1, 1)
            sample_weight = \
                np.squeeze(normalize(sample_weight, axis=0, norm='l1'))

            # X, y, sample_weight = shuffle(X, y, sample_weight,
            #                              random_state=random_state)

            # Boosting step.
            sample_weight, estimator_weight, estimator_error = self._boost(
                iboost,
                X, y,
                sample_weight,
                random_state)

            # Early termination.
            if sample_weight is None:
                break

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

            # Stop if error is zero.
            if estimator_error == 0:
                break

            sample_weight_sum = np.sum(sample_weight)

            # Stop if the sum of sample weights has become non-positive.
            if sample_weight_sum <= 0:
                break

            if iboost < self.n_estimators - 1:
                # Normalize.
                sample_weight /= sample_weight_sum

        return self
 

"""
=============================================================================
#############################################################################
#############################################################################
#############################################################################
"""
dataframe=read_csv('C:/Users/Yousef/Desktop/Boostings/10thyroid/1.csv')

dim=dataframe.shape
array = dataframe.values    
Y = array[:,dim[1]-1]
X = array[:,0:dim[1]-1]

minClass=10000000;
minClass_idx=0;
maxClass=0;
maxClass_idx=0;
from collections import Counter
s=Counter(Y)
for idx, val in enumerate(s):
    if(s[val]<minClass):
        minClass=s[val];
        minClass_idx=val;
    if(s[val]>maxClass):
        maxClass=s[val];
        maxClass_idx=val;
        
n_oversamples = maxClass-minClass;        
        
SEED=42

MAUC=list()
MAUC_Whole=list()
AUCNU=list()
AUCNU_Whole=list()
Accuracy=list()
Accuracy_Whole=list()
Precision=list()
Precision_Whole=list()
F1=list()
F1_Whole=list()
MCC=list()
MCC_Whole=list()
CEN=list()
CEN_Whole=list()
GM=list()
GM_Whole=list()
gm_temp=list()
TPR=list()
TPR_Whole=list()
TNR=list()
TNR_Whole=list()  #Specifity
tnr_temp=list()
G=list()
G_Whole=list()
g_temp=list()
Kappa=list()   #Kappa not computed for each class - It is an overall measurement
Kappa_Whole=list() 
gr_temp = list()
GR=list()
GR_Whole=list()
Recall_MinorityClass=list()
Recall_MinorityClass_Whole=list()
Precision_MinorityClass=list()
Precision_MinorityClass_Whole=list()
F_MinorityClass=list()
F_MinorityClass_Whole=list()
temp_mauc=list()
temp_mcc_ovo=list()
MCC_OVO=list()
MCC_OVO_Whole=list()
temp_mcc_ova=list()
MCC_OVA=list()
MCC_OVA_Whole=list()

for k in range(10):
    SEED=randint(2, 10);
    for i in range(5):
    #for train_inde, test_index in skf.split(X,Y): 
        X_train, X_test, y_train, Y_test = train_test_split(
        X, Y, test_size=0.2,stratify=Y, random_state=i*SEED)
        #X_train, X_test = X[train_index], X[test_index] 
        #y_train, Y_test = Y[train_index], Y[test_index]

        # train model and make predictions
        classification = SMOTEBoost(learning_rate=0.3, n_samples=n_oversamples, n_estimators=100)
        classification.fit(X_train, y_train) 
        y_preds = classification.predict(X_test)    
        cm = ConfusionMatrix(actual_vector=Y_test, predict_vector=y_preds) 
        cm.save_html(os.path.join("","SMOTEBoost_Run="+str(k)+"_Fold="+str(i)))      
        
        Test_ClassSet=list(set(Y_test))
        ova_test=Y_test
        ova_pred=y_preds
        for ind1_c in range(0,len(Test_ClassSet)):
            ova_index1=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]
            ova_index2=[ind for ind, x in enumerate(Y_test) if x != Test_ClassSet[ind1_c]]
            ova_test[ova_index1]=1
            ova_test[ova_index2]=-1
            ova_pred[ova_index1]=1
            ova_pred[ova_index2]=-1
            ova_pred[ova_index2]=0
            temp_mcc_ova.append(matthews_corrcoef(ova_test, ova_pred))
            for ind2_c in range(ind1_c+1,len(Test_ClassSet)):
                Class1_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind1_c]]  
                Class2_index=[ind for ind, x in enumerate(Y_test) if x == Test_ClassSet[ind2_c]]  
                test1=Y_test[Class1_index]
                test2=Y_test[Class2_index]
                pred1=y_preds[Class1_index]
                pred2=y_preds[Class2_index]
                test_classes=np.concatenate((test1,test2), axis=0)
                pred_classes=np.concatenate((pred1,pred2), axis=0) 
                for qq in range(0, len(test_classes)):
                    if(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind1_c]):
                        pred_classes[qq]=Test_ClassSet[ind2_c]
                    elif(test_classes[qq]!=pred_classes[qq] and test_classes[qq]==Test_ClassSet[ind2_c]):
                        pred_classes[qq]=Test_ClassSet[ind1_c]
                temp_mauc.append(roc_auc_score(test_classes, pred_classes))
                temp_mcc_ovo.append(matthews_corrcoef(test_classes, pred_classes))

        for idx, tt in enumerate(cm.F1):            
            gm_temp.append(cm.GM[tt])
            tnr_temp.append(cm.TNR[tt])                       
            g_temp.append(cm.G[tt])
            gr_temp.append(cm.TPR[tt])
            
        
        gm_temp=[0 if v is 'None' else v for v in gm_temp]
        g_temp=[0 if v is 'None' else v for v in g_temp]
        tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
        gr_temp=[0 if v is 'None' else v for v in g_temp]
        MCC_OVA.append(statistics.mean(temp_mcc_ova))
        temp_mcc_ova.clear()
        MCC_OVO.append(statistics.mean(temp_mcc_ovo))
        temp_mcc_ovo.clear()
        MAUC.append(statistics.mean(temp_mauc))
        temp_mauc.clear()
        GM.append(statistics.mean(gm_temp))
        gm_temp.clear()
        G.append(statistics.mean(g_temp))
        g_temp.clear()
        Kappa.append(cm.Kappa)
        AUCNU.append(cm.AUNU)
        F1.append(cm.F1_Macro)
        Precision.append(cm.PPV_Macro)
        TPR.append(cm.TPR_Macro)
        Accuracy.append(cm.Overall_ACC)
        tnr_temp=[0 if v is 'None' else v for v in tnr_temp]
        TNR.append(statistics.mean(tnr_temp))  
        tnr_temp.clear()
        GR.append(gmean(gr_temp))
        gr_temp.clear()
        MCC.append(cm.Overall_MCC)
        CEN.append(cm.Overall_CEN)
        Recall_MinorityClass.append(cm.TPR[minClass_idx])
        Precision_MinorityClass.append(cm.PPV[minClass_idx])
        F_MinorityClass.append(cm.F1[minClass_idx])
    
    MAUC_Whole.append(statistics.mean(MAUC))
    MAUC.clear()
    AUCNU=[0 if v is 'None' else v for v in AUCNU]
    AUCNU_Whole.append(statistics.mean(AUCNU))
    AUCNU.clear()    
    Accuracy_Whole.append(statistics.mean(Accuracy))
    Accuracy.clear()
    Precision=[0 if v is 'None' else v for v in Precision]
    Precision_Whole.append(statistics.mean(Precision))
    Precision.clear()     
    F1_Whole.append(statistics.mean(F1))
    F1.clear()
    GM=[0 if v is 'None' else v for v in GM]
    GM_Whole.append(statistics.mean(GM))
    GM.clear()
    TPR=[0 if v is 'None' else v for v in TPR]
    TPR_Whole.append(statistics.mean(TPR))
    TPR.clear() 
    TNR=[0 if v is 'None' else v for v in TNR]
    TNR_Whole.append(statistics.mean(TNR))
    TNR.clear()
    G=[0 if v is 'None' else v for v in G]
    G_Whole.append(statistics.mean(G))
    G.clear()
    Kappa=[0 if v is 'None' else v for v in Kappa]
    Kappa_Whole.append(statistics.mean(Kappa))
    Kappa.clear()
    MCC_Whole.append(statistics.mean(MCC))
    MCC.clear()
    CEN_Whole.append(statistics.mean(CEN))
    CEN.clear()
    Recall_MinorityClass=[0 if v is 'None' else v for v in Recall_MinorityClass]
    Recall_MinorityClass_Whole.append(statistics.mean(Recall_MinorityClass))
    Precision_MinorityClass=[0 if v is 'None' else v for v in Precision_MinorityClass]
    Precision_MinorityClass_Whole.append(statistics.mean(Precision_MinorityClass))
    F_MinorityClass_Whole.append(statistics.mean(F_MinorityClass))
    Recall_MinorityClass.clear()
    Precision_MinorityClass.clear()
    F_MinorityClass.clear()
    GR_Whole.append(statistics.mean(GR))
    GR.clear()
    MCC_OVO_Whole.append(statistics.mean(MCC_OVO))
    MCC_OVO.clear()
    MCC_OVA_Whole.append(statistics.mean(MCC_OVA))
    MCC_OVA.clear()
    

with open('SMOTEBoost.txt', 'w') as filehandle:
                filehandle.write('AUCNU')
        filehandle.write('\n')
        for listitem in AUCNU_Whole:
            filehandle.write('%s\t' % listitem)   
        filehandle.write('\n\n')
        filehandle.write('MAUC')
        filehandle.write('\n')
        for listitem in MAUC_Whole:
            filehandle.write('%s\t' % listitem) 
        #filehandle.write('\n\n')
        #filehandle.write('ACCURACY')
        #filehandle.write('\n')
        #for listitem in Accuracy_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('Precision')
        #filehandle.write('\n')
        #for listitem in Precision_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('F_measure'),
        filehandle.write('\n')
        for listitem in F1_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('MCC_measure'),
        filehandle.write('\n')
        for listitem in MCC_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('CEN_measure'),
        filehandle.write('\n')
        for listitem in CEN_Whole:
            filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_mean')
        #filehandle.write('\n')
        #for listitem in GM_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TPR')
        #filehandle.write('\n')
        #for listitem in TPR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('TNR')
        #filehandle.write('\n')
        #for listitem in TNR_Whole:
            #filehandle.write('%s\t' % listitem)
        #filehandle.write('\n\n')
        #filehandle.write('G_measure')
        #filehandle.write('\n')
        #for listitem in G_Whole:
            #filehandle.write('%s\t' % listitem)
        filehandle.write('\n\n')
        filehandle.write('Kappa')
        filehandle.write('\n')
        for listitem in Kappa_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('GR')
        filehandle.write('\n')
        for listitem in GR_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Recall_MinorityClass')
        filehandle.write('\n')
        for listitem in Recall_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('Precision_MinorityClass')
        filehandle.write('\n')
        for listitem in Precision_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('F_MinorityClass')
        filehandle.write('\n')
        for listitem in F_MinorityClass_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('MCC_OVO')
        filehandle.write('\n')
        for listitem in MCC_OVO_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')
        filehandle.write('\n')
        filehandle.write('MCC_OVA')
        filehandle.write('\n')
        for listitem in MCC_OVA_Whole:
            filehandle.write('%s\t' % listitem)
        filehandle.write('\n')

        
        
print("%.4f" % statistics.mean(AUCNU_Whole)) #AUC_NU=
print("%.4f" % statistics.mean(MAUC_Whole)) #MAUC
#print("%.4f" % statistics.mean(Accuracy_Whole))
#print("%.4f" % statistics.mean(Precision_Whole))
print("%.4f" % statistics.mean(F1_Whole))
#print("%.4f" % statistics.mean(GM_Whole))
#print("%.4f" % statistics.mean(TPR_Whole)) #Recall
#print("%.4f" % statistics.mean(TNR_Whole)) #Specifity
#print("%.4f" % statistics.mean(G_Whole))
print("%.4f" % statistics.mean(MCC_Whole))
print("%.4f" % statistics.mean(CEN_Whole))
print("%.4f" % statistics.mean(Kappa_Whole))
print("%.4f" % statistics.mean(GR_Whole))
print("%.4f" % statistics.mean(Recall_MinorityClass_Whole))
print("%.4f" % statistics.mean(Precision_MinorityClass_Whole))
print("%.4f" % statistics.mean(F_MinorityClass_Whole))
print("%.4f" % statistics.mean(MCC_OVO_Whole))
print("%.4f" % statistics.mean(MCC_OVA_Whole))
MCC_OVA_Whole.clear()
MCC_Whole.clear()
MAUC_Whole.clear()
F1_Whole.clear
MAUC_Whole.clear()